In [1]:
# -*- coding: utf-8 -*-
"""
Created on May 12, 2024
Last modified on May 14, 2024
@Author: Guan-Fu Liu

To test the MassLifetime.py
"""
import numpy as np
import pandas as pd
from scipy.integrate import quad
import sys
sys.path.insert(0, '/Users/liuguanfu/Workspace/SAS-21/targets/MRK1216/spex/jupyter/IMF/chemevoimf')
from MassLifetime import MassLifetime
import constants
import utils
import h5py

In [2]:
mass_lifetime = MassLifetime()

In [3]:
mass_lifetime.lifetime_to_mass(1e10, 0), utils.lifetime_to_mass(1e10, 0)

(array(0.8748809), array(0.8748809))

In [4]:
file = h5py.File("../inputs/NuPyCEE/isotope_yield_table_portinari98_marigo01_gce_totalyields/yields1.h5", "r")
lifetime = { }
for key in file.keys():
    lifetime[key] = { }
    lifetime[key]['Mini'] = file[key]['MassLifetime'][:,0].astype(float)
    lifetime[key]['lifetime'] = file[key]['MassLifetime'][:,1].astype(float)

mass_lifetime = MassLifetime(lifetime=lifetime)

In [5]:
mass_lifetime.lifetime_to_mass(1e10, 0)

array(0.85937888)

In [6]:
import math

file = open('../../galIMF/yield_tables/rearranged/setllar_lifetime_from_portinari98/portinari98_Z=0.0004.txt','r')
lines = file.readlines()
file.close()
Mini1 = np.array(lines[3].split(), dtype=float)
lifetime1 = np.array(lines[5].split(), dtype=float)

def fucntion_mass_boundary(time, mass_grid_for_lifetime, lifetime):
    # The adopted spline fit of portinari98 lifetime is not monotonic at the massive end. 
    # But this function ensures that lifetime is monotonically smaller for more massive stars.
    mass = mass_grid_for_lifetime
    length_list_lifetime = len(lifetime)
    x = round(length_list_lifetime / 2)
    loop_number_fucntion_mass_boundary = math.ceil(math.log(length_list_lifetime, 2))
    mass_boundary = 10000
    if lifetime[x] == time:
        mass_boundary = mass[x]
    else:
        i = 0
        low = 0
        high = length_list_lifetime
        while i < loop_number_fucntion_mass_boundary:
            if lifetime[x] > time:
                low = x
                x = x + round((high - x) / 2)
            else:
                high = x
                x = x - round((x - low) / 2)
            (i) = (i + 1)
        if x == length_list_lifetime - 1:
            mass_boundary = mass[x]
        else:
            if lifetime[x - 1] > time > lifetime[x]:
                x = x - 1
            mass_boundary = round(mass[x] + (mass[x + 1] - mass[x]) * (lifetime[x] - time) / (
                lifetime[x] - lifetime[x + 1]), 5)
    return mass_boundary


for i in range(1,10):
    print("After %0.2e yr"%(i*1e7))
    print("Stars with mass >=%0.4f Msun will explode in ChemEvoIMF"%utils.lifetime_to_mass(i*1e7, 0))
    print("Stars with mass >=%0.4f Msun will explode in galIMF"%fucntion_mass_boundary(i*1e7, Mini1, lifetime1))
    print("Stars with mass >=%0.4f Msun will explode in new mass lifetime table"%mass_lifetime.lifetime_to_mass(i*1e7, 0))


After 1.00e+07 yr
Stars with mass >=21.1314 Msun will explode in ChemEvoIMF
Stars with mass >=20.2899 Msun will explode in galIMF
Stars with mass >=20.3237 Msun will explode in new mass lifetime table
After 2.00e+07 yr
Stars with mass >=12.5313 Msun will explode in ChemEvoIMF
Stars with mass >=12.0821 Msun will explode in galIMF
Stars with mass >=12.0876 Msun will explode in new mass lifetime table
After 3.00e+07 yr
Stars with mass >=9.7457 Msun will explode in ChemEvoIMF
Stars with mass >=9.3556 Msun will explode in galIMF
Stars with mass >=9.3742 Msun will explode in new mass lifetime table
After 4.00e+07 yr
Stars with mass >=8.2359 Msun will explode in ChemEvoIMF
Stars with mass >=7.9154 Msun will explode in galIMF
Stars with mass >=7.9382 Msun will explode in new mass lifetime table
After 5.00e+07 yr
Stars with mass >=7.2580 Msun will explode in ChemEvoIMF
Stars with mass >=6.9992 Msun will explode in galIMF
Stars with mass >=6.9993 Msun will explode in new mass lifetime table
Afte